# Server Scripts

In [1]:
import os
import six
from google.cloud import translate_v2 as translate
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('data/final/2019-20__match_infos_with_grades_and_summary_translated_cleaned.csv')  

In [3]:
# input from client
score_from_client = [2,0]
possession_from_client = [70,30]
passes_completed_home = [100,300] # range
passes_completed_away = 'not_set'

In [4]:
df.columns

Index(['date', 'name_home_team', 'name_away_team', 'score_home', 'score_away',
       'shots_home', 'shots_away', 'passes_home', 'passes_away',
       'misplaced_passes_home', 'misplaced_passes_away', 'pass_accuracy_home',
       'pass_accuracy_away', 'distance_home', 'distance_away', 'grade',
       'summary_german', 'summary_english', 'coach_home', 'coach_away',
       'interview_home', 'interview_away', 'interview_home_english',
       'interview_away_english'],
      dtype='object')

In [33]:
test = df.loc[df['score_home'] & (df['score_away'] == 2) & (df['shots_home'] > 15) & (df['pass_accuracy_home'] > 0)]
test

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
52,"29.09.2019, 15:30",Fortuna Düsseldorf,SC Freiburg,1,2,18,9,481,408,76,...,114.15,"4,0","Unansehnliche erste 40 Minuten, bessere zweite...","Unsightly first 40 minutes, better second half...",Friedhelm Funkel,Christian Streich,"Wir müssen damit rechnen, im unteren Drittel i...","Es war eng, beide Mannschaften hätten gewinnen...",We have to expect to be in the struggle for su...,"It was close, both teams could have won. We ar..."
55,"05.10.2019, 15:30",Bayern München,TSG Hoffenheim,1,2,20,8,691,394,86,...,126.97,"2,5","Bayern mit der Mehrzahl der Chancen, aber auch...","Bayern with the majority of the chances, but a...",Niko Kovac,Alfred Schreuder,"Der Sieg für Hoffenheim geht in Ordnung, wir h...",Ich bin sehr stolz auf mein Team. Es war das e...,"The win for Hoffenheim is okay, we didn't mana...",I am very proud of my team. It was the first t...
58,"05.10.2019, 15:30",SC Paderborn 07,1. FSV Mainz 05,1,2,16,16,573,301,88,...,120.03,"4,0","Offen, unterhaltsam und mit vielen Torszenen, ...","Open, entertaining and with many goal scenes, ...",Steffen Baumgart,Jan- Lichte,Das war ein verdienter Sieg der Mainzer. Was g...,Wir sind sehr froh über die drei Punkte. Es wa...,That was a deserved victory for the Mainz team...,We are very happy with the three points. It wa...
84,"02.11.2019, 15:30",Bayer 04 Leverkusen,Bor. Mönchengladbach,1,2,25,10,552,370,98,...,121.58,"3,0","Spannende Partie, in der aber beide Mannschaft...","Exciting game, in which both teams did not act...",Peter Bosz,Marco Rose,Die erste Hälfte war die schlechteste Hälfte s...,Wir sind natürlich mit dem 2:1 in Leverkusen s...,The first half was the worst half since I've b...,We are of course very happy with the 2-1 win i...
102,"23.11.2019, 15:30",Werder Bremen,FC Schalke 04,1,2,18,11,466,478,110,...,118.34,"3,5",Eine über weite Strecken taktisch geprägte Par...,"A largely tactical game. In the end exciting, ...",Florian Kohfeldt,David Wagner,"Wir müssen uns den Vorwurf gefallen lassen, da...",In der ersten Halbzeit war es ein Spiel mit se...,We have to put up with the accusation that we ...,In the first half it was a game with very few ...
113,"30.11.2019, 18:30",Bayern München,Bayer 04 Leverkusen,1,2,23,11,708,241,90,...,124.62,"1,5","Rasse, Klasse, Tempo, Spannung: ein Spitzenspi...","Race, class, speed, tension: a top game with h...",Hans- Flick,Peter Bosz,In der ersten Halbzeit haben wir zweimal nicht...,"Wir haben Glück gehabt. Das, was uns in der le...",We didn't pay attention twice in the first hal...,We were lucky. What we've been missing in the ...
156,"18.01.2020, 15:30",1. FSV Mainz 05,SC Freiburg,1,2,24,13,458,331,106,...,118.86,"2,5","Eine spannende Partie auf gehobenem Niveau, di...","An exciting game at a high level, which, despi...",Achim Beierlorzer,Christian Streich,"Die Effizienz des SC, die würden wir uns wünsc...",Wir hatten im richtigen Moment das nötige Glüc...,"The efficiency of the SC, that is what we woul...",We were lucky at the right moment and were the...
164,"25.01.2020, 15:30",VfL Wolfsburg,Hertha BSC,1,2,20,9,546,381,107,...,115.22,"4,0","Viel Leerlauf, erst nach der Pause abwechslung...","Lots of idle time, only after the break more v...",Oliver Glasner,Jürgen Klinsmann,"Wir haben viel Aufwand betrieben, um gegen den...",Für uns ist es ein unglaublich wertvoller Drei...,We went to great lengths to score a goal again...,It's an incredibly valuable threesome for us. ...
193,"15.02.2020, 15:30",SC Paderborn 07,Hertha BSC,1,2,21,16,406,369,97,...,115.84,"4,0","Zähe erste Halbzeit, im zweiten Durchgang etwa...","Tough first half, in the second half a little ...",Steffen Baumgart,Alexander Nouri,Wir hatten uns das anders vorgestellt. Jetzt i...,Es war das erwarte